In [4]:
import sys
import argparse
import numpy as np
import cv2
import datetime
import time
import pytz
import json

from pipelines.document_structurization import DocumentStructurization
from modules.file_loading import load_document, load_whole_pdf

import pdfplumber
from pathlib import Path
import os
import logging
import pandas as pd
from langchain_core.documents import Document
from langchain_core.load.dump import dumps as lang_dumps


/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
[t for t in pytz.all_timezones if t.startswith('America/Mex')]

['America/Mexico_City']

In [6]:
root_string = '/home/camilo/Documents/00-Conabio/by_species/' #Casa
root_string = '/home/camilo/Documents/by_species/' #Conabio


root_path = Path(root_string)

processed_list=['Leptonycteris yerbabuenae', 'Leptonycteris nivalis', 'Melipona beecheii'] 
species_subfolders = os.listdir(root_path)
pending_process_species = list(set(species_subfolders)-set(processed_list))
pending_process_species

['test species A', 'test species B']

#### Funcitons

In [7]:
def check_species_processed(root_path, 
                            processed_list=['Leptonycteris yerbabuenae', 'Leptonycteris nivalis', 'Melipona beecheii']):
    species_subfolders = os.listdir(root_path)
    pending_process_species = list(set(species_subfolders)-set(processed_list))
    return pending_process_species 


In [8]:
def get_list_paths(root_path, 
               input_subfolder_sufix="_bibliografía", 
               output_subfolder = "output"
               ):

    path_dics = []
    
    pending_process_species = check_species_processed(root_path)

    for species_folder in pending_process_species:
        species_folder_path = Path(root_path,species_folder)
        file_name_list = os.listdir(Path(root_path,species_folder,species_folder+'_bibliografía'))
        for file_name in file_name_list:
            path_dics.append(
                dict(
                    file_input_path = Path(species_folder_path,str(species_folder_path.name)+input_subfolder_sufix,file_name),
                    folder_output_path = Path(species_folder_path,output_subfolder),
                    species_folder = species_folder,
                    file_name = file_name
                )
            )
    return path_dics

list_paths = get_list_paths(root_path)

In [9]:
def load_whole_pdf(pdf_path):

    # initialization
    image_list = []

    # read PDF file (load all pages in the PDF file)
    name = pdf_path.lower()
    if name.endswith('.pdf'):
        with pdfplumber.open(pdf_path) as pdf:
            page_count = len(pdf.pages)
            for page_index in range(page_count):  # traverse all pages
                page = pdf.pages[page_index]  # select the current page
                page_image = page.to_image(resolution=150) # convert the page to image by default (20230815)
                image = cv2.cvtColor(np.array(page_image.original), cv2.COLOR_RGB2BGR)

                image_list.append(image)

            pdf.close()

    return image_list

In [10]:
def whole_pdf_conversion_example(image_list):

    # configure
    configs = dict()
    
    layout_analysis_configs = dict()
    layout_analysis_configs['from_modelscope_flag'] = False
    layout_analysis_configs['model_path'] = '/home/DocXLayout_231012.pth'  # note that: currently the layout analysis model is NOT from modelscope
    configs['layout_analysis_configs'] = layout_analysis_configs
    
    text_detection_configs = dict()
    text_detection_configs['from_modelscope_flag'] = True
    text_detection_configs['model_path'] = 'damo/cv_resnet18_ocr-detection-line-level_damo'
    configs['text_detection_configs'] = text_detection_configs

    text_recognition_configs = dict()
    text_recognition_configs['from_modelscope_flag'] = True
    text_recognition_configs['model_path'] = 'damo/cv_convnextTiny_ocr-recognition-document_damo'  # alternatives: 'damo/cv_convnextTiny_ocr-recognition-scene_damo', 'damo/cv_convnextTiny_ocr-recognition-general_damo', 'damo/cv_convnextTiny_ocr-recognition-handwritten_damo' 
    configs['text_recognition_configs'] = text_recognition_configs

    formula_recognition_configs = dict()
    formula_recognition_configs['from_modelscope_flag'] = False
    formula_recognition_configs['image_resizer_path'] = '/home/LaTeX-OCR_image_resizer.onnx'
    formula_recognition_configs['encoder_path'] = '/home/LaTeX-OCR_encoder.onnx'
    formula_recognition_configs['decoder_path'] = '/home/LaTeX-OCR_decoder.onnx'
    formula_recognition_configs['tokenizer_json'] = '/home/LaTeX-OCR_tokenizer.json'
    configs['formula_recognition_configs'] = formula_recognition_configs

    # initialize
    document_structurizer = DocumentStructurization(configs)

    # run
    final_result = []
    page_index = 0
    for image in image_list:
        result = document_structurizer(image)

        page_info = {'page': page_index, 'information': result}
        final_result.append(page_info)

        page_index = page_index + 1

    if True:
        print (final_result)

    # release
    document_structurizer.release()

    return final_result

In [11]:
def loop_OCR(root_path, max_files = 100): 
    '''
    Expected folder Structure :

    root_folder/
        species A/
            species A_bibiliografía/
                file_A1.pdf
                file_A2.pdf
            output/
                Doc_A1
                Doc_A2
    
    Note: The subfolders under root_folde define the species-name. 
    '''
    list_paths = get_list_paths(root_path)
    final_results = []

    for file_path_dict in list_paths[0:max_files]:
        pdf_path = file_path_dict.get('file_input_path')
        output_folder = file_path_dict.get('folder_output_path')
        if not os.path.exists(output_folder):
            print("Creating output folder in : {output_folder.parent.name}")
            os.mkdir(output_folder)

        image_list = load_whole_pdf(str(pdf_path))

        try:
            final_result = whole_pdf_conversion_example(image_list)
            
            
        
        except:
            final_result = {'error_species':str(file_path_dict.get('species_folder')), 
                            'error_file': str(file_path_dict.get('file_input_path'))
                            }
            logging.error("Custom_error_msg", exc_info=True)
        
        final_results.append(final_result)
    
    return final_results

In [12]:
def res_to_df_chunks(final_result: list, filter_criteria = ['plain text']) -> Document:
    # 1. to DataFrame and "Explode" for relevant columns
    # 2. Select type of objects, default 'plain text' category
    # 3. Group text into region-polygon-chunks

    
    try: 
        df = pd.DataFrame(final_result)
        cols = ['information','text_list']

        for col in cols:
            #print(col)
            df = df.explode(col).reset_index(drop=True)
            df = df.drop(columns=[col]).join(df[col].apply(pd.Series), rsuffix=f".{col}")

        df['content'] = df['content'].apply(lambda x: x[0] if isinstance(x,list) else '')
        df['region_poly'] = df['region_poly'].apply(lambda x: tuple(x)) # lists are mutable 
        df['content_type'] = df ['content'].apply(lambda x: type(x).__name__)
        
        # 2
        df = df[df.category_name.apply(lambda x: x in filter_criteria)]

        # 3
        aggr_level = ['page', 'region_poly']
        grouped_df = df.groupby(aggr_level)
        # Concatenate the content of the 'content' column with spaces
        df_agg = df.groupby(aggr_level)['content'].apply(lambda x: ' '.join(x)).reset_index()
    
    except:
        return (final_result)

        
    return df_agg

In [13]:
def df_to_doc(df_agg, file_path_dict):
    """Converts a Pandas DataFrame to a list of LangChain Documents with enhanced data handling.

    Args:
        df_agg (pandas.DataFrame): The DataFrame containing the data to be converted.
        file_path_dict (dict): A dictionary containing file paths for various purposes.

    Returns:
        list: A list of LangChain Document objects with page_content and metadata.
    """

    documents = []
    for _, row in df_agg.iterrows():
        metadata = {
            "page": row.get("page"),  # Use get() for potential missing values
            "file_name": file_path_dict.get("file_name"),
            "region_poly": row.get("region_poly"),
            "input_file": str(file_path_dict.get("file_input_path")),
            "species_folder": file_path_dict.get("species_folder"),
            "output_folder": str(Path(file_path_dict.get("folder_output_path"))),  # Convert Path to string for JSON
            "output_file": str(Path(file_path_dict.get("folder_output_path"), file_path_dict.get("file_name").split(".")[0] + ".json"))  # Use get() and string conversion
        }

        # Remove potential None values from metadata
        metadata = {key: value for key, value in metadata.items() if value is not None}

        documents.append(Document(page_content=row.get("content"), metadata=metadata))

    return documents

In [121]:
def df_to_dict(df_agg, file_path_dict):
    documents = [ 
        dict(
            page_content=row.get("content"),
            metadata={
                "page": row.get("page"),  # Use get() for potential missing values
                "file_name": file_path_dict.get("file_name"),
                "region_poly": row.get("region_poly"),
                "input_file": str(file_path_dict.get("file_input_path")),
                "species_folder": file_path_dict.get("species_folder"),
                "output_folder": str(Path(file_path_dict.get("folder_output_path"))),  # Convert Path to string for JSON
                "output_file": str(Path(file_path_dict.get("folder_output_path"), file_path_dict.get("file_name").split(".")[0] + ".json"))  # Use get() and string conversion
            }
        )
        for _, row in df_agg.iterrows()
    ]

    return documents#[0]

In [15]:
from typing import Union

def dfs_to_dicts(dfs: list[Union[dict, pd.DataFrame]], list_paths) -> list:
    
    all_dicts = []
    for df,path_dict in zip(dfs,list_paths):    
        if isinstance(df, pd.DataFrame):
            all_dicts.append(df_to_dict(df, path_dict))
        else: 
            all_dicts.append(df)

    return all_dicts

In [16]:
def new_list_paths(root_path= root_path, pattern='*/output*.json'):
    return list(root_path.glob(pattern=pattern))

In [17]:
def write_to_json(dfs:list,list_paths):
     for res, path_dic in  zip(dfs,list_paths):
          output_folder = path_dic.get('folder_output_path')
          output_file_path = Path(output_folder,
                                   path_dic.get('file_name').split(".")[0]+".json")
               
          # if it is a DataFrame, result finished correctly
          if isinstance(res, pd.DataFrame):
               list_of_docs = df_to_doc(res,path_dic)
               if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
               for chunk in list_of_docs:
                         with open (output_file_path,"a") as fp:
                              json.dump(chunk.dict(),fp)
                              #lang_dumps(list_of_docs, fp)
          else:
               with open ('./log_errors',"a") as fp:
                              try:
                                   json.dump(res,fp)
                              except:
                                   print("TypeError: Object of type PosixPath is not JSON serializable")


### ALL TOGETHER NOW!

In [18]:
# Step 1: Generate list of dictionaries with multiple paths
# All items
list_paths = get_list_paths(root_path)

list_paths[0], \
list_paths[0].keys(), \
list_paths[0].get('folder_output_path').parent.name

({'file_input_path': PosixPath('/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Arita, 1991.pdf'),
  'folder_output_path': PosixPath('/home/camilo/Documents/by_species/test species A/output'),
  'species_folder': 'test species A',
  'file_name': 'Arita, 1991.pdf'},
 dict_keys(['file_input_path', 'folder_output_path', 'species_folder', 'file_name']),
 'test species A')

In [19]:
# Step 2: Run the OCR. 
# uses: get_list_paths, loop_OCR
# All items
final_results = loop_OCR(root_path, max_files = 3)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /home/camilo/Documents/01-tutoriales/docxchain/modules/../DocXLayout/../../exp/ctdet_subfield/default
heads {'hm': 11, 'cls': 4, 'ftype': 3, 'wh': 8, 'hm_sub': 2, 'wh_sub': 8, 'reg': 2, 'reg_sub': 2}
Namespace(task='ctdet_subfield', dataset='huntie', test=False, data_src='default', exp_id='default', vis_corner=0, convert_onnx=0, onnx_path='auto', debug=0, load_model='/home/DocXLayout_231012.pth', resume=False, gpus=[0], num_workers=16, not_cuda_benchmark=False, seed=317, print_iter=0, hide_data_time=False, save_all=False, metric='loss', vis_thresh=0.3, nms_thresh=0.3, corner_thresh=0.3, debugger_theme='white', arch='dlav0subfield_34', head_conv=256, down_ratio=4, input_res=768, input_h=768, input_w=768, lr=0.000125, lr_step=[80], NotFixList='', num_epochs=90, batch_size=32, master_batch_size=32, num_iters=-1, val_intervals=5, trainval=False, negative=False, adamW=False, save_dir='/home/camilo/Documents/01-tutoria

/home/camilo/Documents/01-tutoriales/docxchain/modules/../DocXLayout/models/model.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, ma

09/19/2024 10:33:31 - WARNING - tensorflow -   From /home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/modelscope/utils/device.py:60: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2024-09-19 10:33:30.973794: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-19 10:33:30.974674: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-09-19 10:33:31.015657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/t

09/19/2024 10:33:33 - WARNING - tensorflow -   From /home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/modelscope/pipelines/cv/ocr_utils/ops.py:744: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


...

09/19/2024 10:36:06 - ERROR - root -   Custom_error_msg
Traceback (most recent call last):
  File "/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/rapid_latex_ocr/main.py", line 102, in __call__
    resizered_img = self.loop_image_resizer(img)
  File "/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/rapid_latex_ocr/main.py", line 126, in loop_image_resizer
    input_image = self.pre_pro.minmax_size(pad_img).convert("RGB")
  File "/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/rapid_latex_ocr/utils.py", line 78, in minmax_size
    img = img.resize(size.astype(int), Image.BILINEAR)
  File "/home/camilo/Documents/01-tutoriales/docxchain/env_docx/lib/python3.10/site-packages/PIL/Image.py", line 2297, in resize
    if self.size == size and box == (0, 0) + self.size:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

The above ex

2024-09-19 10:36:13,032 - modelscope - WARNING - Model revision not specified, use revision: v1.0.1
2024-09-19 10:36:13,590 - modelscope - INFO - initiate model from /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo
2024-09-19 10:36:13,592 - modelscope - INFO - initiate model from location /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo.
2024-09-19 10:36:13,601 - modelscope - WARNING - No preprocessor field found in cfg.
2024-09-19 10:36:13,602 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-09-19 10:36:13,604 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo'}. trying to build by task and model information.
2024-09-19 10:36:13,606 - modelscope - WARNING - Find task: ocr-detection, model type: None. Insu

09/19/2024 10:36:16 - INFO - tensorflow -   Restoring parameters from /home/camilo/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo/tf_ckpts/checkpoint-80000


2024-09-19 10:36:20,120 - modelscope - WARNING - Model revision not specified, use revision: v2.4.0
2024-09-19 10:36:20,619 - modelscope - INFO - initiate model from /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-19 10:36:20,622 - modelscope - INFO - initiate model from location /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo.
2024-09-19 10:36:20,629 - modelscope - INFO - initialize model from /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-19 10:36:20,921 - modelscope - INFO - loading model from dir /home/camilo/.cache/modelscope/hub/damo/cv_convnextTiny_ocr-recognition-document_damo
2024-09-19 10:36:20,957 - modelscope - INFO - loading model done
2024-09-19 10:36:22.814358: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retu

[{'page': 0, 'information': [{'category_index': 0, 'category_name': 'title', 'region_poly': [191, 405, 1053, 405, 1053, 573, 191, 573], 'text_list': [{'position': [193, 403, 1044, 404, 1044, 459, 193, 459], 'content': ['PRODUCCION DEL ACHIOTE']}, {'position': [334, 508, 905, 509, 904, 571, 334, 570], 'content': ['(Bixa Orellana L.)"']}]}, {'category_index': 0, 'category_name': 'title', 'region_poly': [314, 276, 925, 276, 925, 338, 314, 338], 'text_list': [{'position': [328, 276, 926, 277, 926, 332, 327, 330], 'content': ['“MANUAL PARA LA']}]}, {'category_index': 1, 'category_name': 'figure', 'region_poly': [257, 701, 979, 701, 979, 1468, 257, 1468], 'text_list': []}]}, {'page': 1, 'information': [{'category_index': 0, 'category_name': 'title', 'region_poly': [286, 146, 952, 146, 952, 189, 286, 189], 'text_list': [{'position': [282, 145, 955, 145, 955, 187, 282, 186], 'content': ['UNIVERSIDAD VERACRUZANA']}]}, {'category_index': 0, 'category_name': 'title', 'region_poly': [222, 954, 102

In [21]:
len(final_results),\
final_results[2][0].keys(),
final_results[1]

{'error_species': 'test species A',
 'error_file': '/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Pourshoushtari, 2019.pdf'}

In [22]:
len(final_results),final_results[2][0].keys(), final_results[2][0].get('information')

(3,
 dict_keys(['page', 'information']),
 [{'category_index': 0,
   'category_name': 'title',
   'region_poly': [191, 405, 1053, 405, 1053, 573, 191, 573],
   'text_list': [{'position': [193, 403, 1044, 404, 1044, 459, 193, 459],
     'content': ['PRODUCCION DEL ACHIOTE']},
    {'position': [334, 508, 905, 509, 904, 571, 334, 570],
     'content': ['(Bixa Orellana L.)"']}]},
  {'category_index': 0,
   'category_name': 'title',
   'region_poly': [314, 276, 925, 276, 925, 338, 314, 338],
   'text_list': [{'position': [328, 276, 926, 277, 926, 332, 327, 330],
     'content': ['“MANUAL PARA LA']}]},
  {'category_index': 1,
   'category_name': 'figure',
   'region_poly': [257, 701, 979, 701, 979, 1468, 257, 1468],
   'text_list': []}])

In [23]:
df = res_to_df_chunks(final_results[0], filter_criteria = ['plain text'])
dfs = [res_to_df_chunks(df,filter_criteria = ['plain text']) for df in final_results]

In [24]:
dfs[1]

{'error_species': 'test species A',
 'error_file': '/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Pourshoushtari, 2019.pdf'}

In [84]:
# jsons (Results) to dictionaries
# uses: get_list_paths, df_to_dict
# One item
list_paths = get_list_paths(root_path)
path_dict = list_paths[0]
dicts = df_to_dict(df,path_dict)
all_dicts = dfs_to_dicts(dfs,list_paths)


In [85]:
for obj in all_dicts:
    print(type(obj))

<class 'list'>
<class 'dict'>
<class 'list'>


In [86]:
all_dicts

[[{'page_content': 'Some of the authors ofthis publication are also working on these related projects:',
   'metadata': {'page': 0,
    'file_name': 'Arita, 1991.pdf',
    'region_poly': (81, 657, 570, 657, 570, 672, 81, 672),
    'input_file': '/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Arita, 1991.pdf',
    'species_folder': 'test species A',
    'output_folder': '/home/camilo/Documents/by_species/test species A/output',
    'output_file': '/home/camilo/Documents/by_species/test species A/output/Arita, 1991.json'}},
  {'page_content': 'Article in Joumal of Ma nalogy·No vember 1991',
   'metadata': {'page': 0,
    'file_name': 'Arita, 1991.pdf',
    'region_poly': (86, 302, 372, 302, 372, 322, 86, 322),
    'input_file': '/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Arita, 1991.pdf',
    'species_folder': 'test species A',
    'output_folder': '/home/camilo/Documents/by_species/test species A/output',
    'output_file':

In [125]:
#NEW 
dfs = [res_to_df_chunks(df) for df in final_results]

def write_to_json(dfs:list,list_paths, mode = 'override'):
     for res, path_dic in  zip(dfs,list_paths):
          output_folder = path_dic.get('folder_output_path')
          output_file_path = Path(output_folder,
                                   path_dic.get('file_name').split(".")[0]+".json")
               
          
          if mode == "skip" and os.path.exists(output_file_path):
                continue  # Skip existing file
          
          # if it is a DataFrame, result finished correctly
          if isinstance(res, pd.DataFrame):
               list_of_dicts = df_to_dict(res,path_dic)
               if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
               
               with open (output_file_path,"w") as fp:
                    json.dump(list_of_dicts,fp, indent=2)
          else:
               with open ('./log_errors',"a") as fp:
                              try:
                                   json.dump(res,fp)
                              except:
                                   print("TypeError: Object of type PosixPath is not JSON serializable")

In [123]:
df_to_dict(dfs[0],list_paths[0])

[{'page_content': 'Some of the authors ofthis publication are also working on these related projects:',
  'metadata': {'page': 0,
   'file_name': 'Arita, 1991.pdf',
   'region_poly': (81, 657, 570, 657, 570, 672, 81, 672),
   'input_file': '/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Arita, 1991.pdf',
   'species_folder': 'test species A',
   'output_folder': '/home/camilo/Documents/by_species/test species A/output',
   'output_file': '/home/camilo/Documents/by_species/test species A/output/Arita, 1991.json'}},
 {'page_content': 'Article in Joumal of Ma nalogy·No vember 1991',
  'metadata': {'page': 0,
   'file_name': 'Arita, 1991.pdf',
   'region_poly': (86, 302, 372, 302, 372, 322, 86, 322),
   'input_file': '/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Arita, 1991.pdf',
   'species_folder': 'test species A',
   'output_folder': '/home/camilo/Documents/by_species/test species A/output',
   'output_file': '/home/camilo/D

In [116]:
list_paths[0]

{'file_input_path': PosixPath('/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Arita, 1991.pdf'),
 'folder_output_path': PosixPath('/home/camilo/Documents/by_species/test species A/output'),
 'species_folder': 'test species A',
 'file_name': 'Arita, 1991.pdf'}

In [136]:
file_path='/home/camilo/Documents/by_species/test species B/output/Narciso-Reyes, 2003.json'
file_path = "/home/camilo/Documents/by_species/test.json" 
list_of_dicts = df_to_dict(dfs[0],list_paths[0])

with open (file_path,"w") as fp:
    json.dump(list_of_dicts,fp, indent=2)

with open (file_path,"r") as fp:
    json_loaded = json.load(fp)
json_loaded[0:2]

[{'page_content': 'Some of the authors ofthis publication are also working on these related projects:',
  'metadata': {'page': 0,
   'file_name': 'Arita, 1991.pdf',
   'region_poly': [81, 657, 570, 657, 570, 672, 81, 672],
   'input_file': '/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Arita, 1991.pdf',
   'species_folder': 'test species A',
   'output_folder': '/home/camilo/Documents/by_species/test species A/output',
   'output_file': '/home/camilo/Documents/by_species/test species A/output/Arita, 1991.json'}},
 {'page_content': 'Article in Joumal of Ma nalogy·No vember 1991',
  'metadata': {'page': 0,
   'file_name': 'Arita, 1991.pdf',
   'region_poly': [86, 302, 372, 302, 372, 322, 86, 322],
   'input_file': '/home/camilo/Documents/by_species/test species A/test species A_bibliografía/Arita, 1991.pdf',
   'species_folder': 'test species A',
   'output_folder': '/home/camilo/Documents/by_species/test species A/output',
   'output_file': '/home/camilo/D

In [87]:
# ANTES
dfs = [res_to_df_chunks(df) for df in final_results]

def write_to_json2(dfs:list,list_paths, mode='override'):
    print(os.getcwd())
    for res, path_dic in  zip(dfs,list_paths):
        output_folder = path_dic.get('folder_output_path')
        output_file_path = Path(output_folder,
                                    path_dic.get('file_name').split(".")[0]+".json")
                
        # if it is a DataFrame, result finished correctly
        if isinstance(res, pd.DataFrame):
            list_of_docs = df_to_doc(res,path_dic)
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            
            if mode == "skip" and os.path.exists(output_file_path):
                continue  # Skip existing file
            
            chunks = [json.dumps(chunk.dict(), indent=2) for chunk in list_of_docs]
            with open (output_file_path,"w") as fp:
                fp.write(','.join(chunks))

        else:
            with open("./log_errors", "a") as fp:
                try:
                    json.dump(res, fp)
                    fp.write(",\n".join(chunks))

                except Exception as e:  # Catch specific exception
                    print(f"Error serializing object: {e}")

In [126]:

#write_to_json(dfs,list_paths)

In [111]:
file_path='/home/camilo/Documents/by_species/test species B/output/Narciso-Reyes, 2003.json'

def read_multiple_jsons(file_path):
    with open(file_path, 'r')as f:
        json_string = f.read()
        json_objects = json.loads('[' + json_string[:-2].replace(',', ', ') + ']')
        return json_objects

data = read_multiple_jsons(file_path)


In [115]:
data[0]

{'id': None,
 'metadata': {'page': 1,
  'file_name': 'Narciso-Reyes,  2003.pdf',
  'region_poly': [261, 1275, 990, 1275, 990, 1382, 261, 1382],
  'input_file': '/home/camilo/Documents/by_species/test species B/test species B_bibliografía/Narciso-Reyes,  2003.pdf',
  'species_folder': 'test species B',
  'output_folder': '/home/camilo/Documents/by_species/test species B/output',
  'output_file': '/home/camilo/Documents/by_species/test species B/output/Narciso-Reyes,  2003.json'},
 'page_content': 'PROFESOR DE EXPERIENCIA RECEPCIONAL: ING. GABRIEL MAY MORA',
 'type': 'Document'}

In [ ]:
def load_to_bd(root_path, method="output", bd_creds= ["user", "password"]):
    
    # generate a list of dictoniaries of paths with custom ouput folders
    json_method_paths = list(root_path.rglob('*/'+method+'/*.json'))

    for path in json_method_paths:

        with open (path,"r") as fp:
            placeholder = json.load(fp)
            insert_sql = f"INSERT INTO {table_name} VALUES ({placeholders})"

        
        
    
        



In [146]:
method = 'output'
list(root_path.rglob('*/'+'output'+'/*.json'))

[PosixPath('/home/camilo/Documents/by_species/test species B/output/Narciso-Reyes, 2003_original.json'),
 PosixPath('/home/camilo/Documents/by_species/test species B/output/Paxton et al.json'),
 PosixPath('/home/camilo/Documents/by_species/test species B/output/Genaro, 2018.json'),
 PosixPath('/home/camilo/Documents/by_species/test species B/output/Narciso-Reyes, 2003.json'),
 PosixPath('/home/camilo/Documents/by_species/test species A/output/Pourshoushtari, 2019.json'),
 PosixPath('/home/camilo/Documents/by_species/test species A/output/Arita, 1991.json')]